In [1]:
import pandas as pd
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, cross_val_predict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score,recall_score, precision_score, accuracy_score, f1_score, roc_curve,auc,precision_recall_curve
import numpy as np
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/PNI_001_cleaned.csv')
X = df.drop(['PNI','ID'],axis = 1)
y = df['PNI']

## Perform 10 random (80/20) splits and report AUC for PNI classifier

### shuffle before splitting for more randomness

In [3]:
X, y = shuffle(X, y, random_state=0)

In [4]:
roc_auc_list = []
for i in range(10):
    # split dataset randomly (change random state parameter every iteration)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = i, test_size = .2)
    
    # fit model and output prediction probabilities for class 1 
    rf_model = RandomForestClassifier(n_estimators=100, 
                                      max_depth=25, 
                                      min_samples_leaf=3, 
                                      verbose=0, 
                                      random_state = 1)
    rf_model.fit(X_train,y_train)
    
    y_pred_proba = rf_model.predict_proba(X_test)[:,1]
    
    # generate roc auc score and append to the list 
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc_auc_list.append(roc_auc)

In [5]:
roc_auc_list

[0.778446261682243,
 0.7955356044766942,
 0.7620929683813119,
 0.7409467309356075,
 0.7684190809190808,
 0.8216149068322982,
 0.7684227330779055,
 0.7911845401407611,
 0.7920454545454545,
 0.7709165834165834]

## report AUC statistics

In [6]:
print('mean auc for 10 folds (all features) is {:.3f}'.format(np.mean(roc_auc_list)))
print('std auc for 10 folds (all features) is {:.3f}'.format(np.std(roc_auc_list)))
print('max auc for 10 folds (all features) is {:.3f}'.format(np.max(roc_auc_list)))
print('min auc for 10 folds (all features) is {:.3f}'.format(np.min(roc_auc_list)))

mean auc for 10 folds is 0.779
std auc for 10 folds is 0.021
max auc for 10 folds is 0.822
min auc for 10 folds is 0.741


## Obtain index of best fold

In [7]:
print('best fold index is {}'.format(np.argmax(roc_auc_list)))

best fold index is 5


## Refit best fold and validate performance

In [8]:
# split dataset randomly (change random state parameter every iteration)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5, test_size = .2)

# fit model and output prediction probabilities for class 1 
rf_model = RandomForestClassifier(n_estimators=100, 
                                  max_depth=25, 
                                  min_samples_leaf=3, 
                                  verbose=0, 
                                  random_state = 1)
rf_model.fit(X_train,y_train)

y_pred_proba = rf_model.predict_proba(X_test)[:,1]

# generate roc auc score and append to the list 
roc_auc = roc_auc_score(y_test, y_pred_proba)
print('best fold roc auc score is {:.3f}'.format(roc_auc))

best fold roc auc score is 0.822


## Output feature importance

In [9]:
df_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'feature_importance': rf_model.feature_importances_
})

In [10]:
df_feature_importance = df_feature_importance.sort_values(by = 'feature_importance', ascending = False)

In [11]:
df_feature_importance.head(30)

,feature,feature_importance
676,PRSS23,0.010923
977,VPS45,0.010585
983,WDR89,0.008390
35,ANKUB1,0.008278
701,RALY,0.006719
388,HSPD1,0.006644
659,PPP2R2DP1,0.005872
185,CNOT6,0.005806
519,MIR4797,0.005749
891,TEX38,0.005330


In [12]:
df_feature_importance.to_csv('../data/pni_follow_up_analysis_feature_importance.csv',index = False)

# Fit model (10 folds - 80/20 split) with only top 25 features

In [16]:
top_25_features = list(df_feature_importance.head(25)['feature'].values)

In [18]:
print(top_25_features)

['PRSS23', 'VPS45', 'WDR89', 'ANKUB1', 'RALY', 'HSPD1', 'PPP2R2DP1', 'CNOT6', 'MIR4797', 'TEX38', 'RPUSD4', 'TMEM79', 'TLR10', 'C4orf45', 'EZH2P1', 'RPL23AP57', 'APOBEC3B', 'SH2B3', 'KRTAP5-2', 'MGMT', 'FBXO43', 'ABHD16B', 'PRRC1', 'RN7SL88P', 'CATSPER1']


In [19]:
X_25 = X[top_25_features]

### confirm that there are only 25 columns

In [25]:
X_25.shape

(1334, 25)

## fit 10 folds for the 25 column data

In [23]:
roc_auc_list_25 = []
for i in range(10):
    # split dataset randomly (change random state parameter every iteration)
    X_train_25, X_test_25, y_train, y_test = train_test_split(X_25, y, random_state = i, test_size = .2)
    
    # fit model and output prediction probabilities for class 1 
    rf_model_25 = RandomForestClassifier(n_estimators=100, 
                                      max_depth=25, 
                                      min_samples_leaf=3, 
                                      verbose=0, 
                                      random_state = 1)
    rf_model_25.fit(X_train_25,y_train)
    
    y_pred_proba_25 = rf_model_25.predict_proba(X_test_25)[:,1]
    
    # generate roc auc score and append to the list 
    roc_auc_25 = roc_auc_score(y_test, y_pred_proba_25)
    roc_auc_list_25.append(roc_auc_25)

In [24]:
roc_auc_list_25

[0.772196261682243,
 0.789755257655885,
 0.7763685700802266,
 0.7484859720677296,
 0.7694805194805195,
 0.782111801242236,
 0.7796934865900383,
 0.7676845997852798,
 0.7852941176470588,
 0.780531968031968]

### report auc stats for 25 feature model

In [27]:
print('mean auc for 10 folds (25 features) is {:.3f}'.format(np.mean(roc_auc_list_25)))
print('std auc for 10 folds (25 features) is {:.3f}'.format(np.std(roc_auc_list_25)))
print('max auc for 10 folds (25 features) is {:.3f}'.format(np.max(roc_auc_list_25)))
print('min auc for 10 folds (25 features) is {:.3f}'.format(np.min(roc_auc_list_25)))

mean auc for 10 folds (25 features) is 0.775
std auc for 10 folds (25 features) is 0.011
max auc for 10 folds (25 features) is 0.790
min auc for 10 folds (25 features) is 0.748


# Fit model (10 folds - 80/20 split) with only top 50 features

In [28]:
top_50_features = list(df_feature_importance.head(50)['feature'].values)

In [29]:
print(top_50_features)

['PRSS23', 'VPS45', 'WDR89', 'ANKUB1', 'RALY', 'HSPD1', 'PPP2R2DP1', 'CNOT6', 'MIR4797', 'TEX38', 'RPUSD4', 'TMEM79', 'TLR10', 'C4orf45', 'EZH2P1', 'RPL23AP57', 'APOBEC3B', 'SH2B3', 'KRTAP5-2', 'MGMT', 'FBXO43', 'ABHD16B', 'PRRC1', 'RN7SL88P', 'CATSPER1', 'C1QL4', 'PTGES3L-AARSD1', 'NOS2', 'CRYBG2', 'SRGN', 'PDLIM2', 'FGFRL1', 'H1-6', 'UGT3A2', 'EVC2', 'SYNC', 'PNRC2', 'FCRL2', 'MBLAC2', 'CUL4B', 'SULT1A1', 'MAGI2', 'PPP4R1', 'SNX29P1', 'UTP3', 'ZNF547', 'SEMA4B', 'RNA5SP466', 'TDRD6', 'GPR88']


In [30]:
X_50 = X[top_50_features]

### confirm that there are only 50 columns

In [31]:
X_50.shape

(1334, 50)

## fit 10 folds for the 50 column data

In [32]:
roc_auc_list_50 = []
for i in range(10):
    # split dataset randomly (change random state parameter every iteration)
    X_train_50, X_test_50, y_train, y_test = train_test_split(X_50, y, random_state = i, test_size = .2)
    
    # fit model and output prediction probabilities for class 1 
    rf_model_50 = RandomForestClassifier(n_estimators=100, 
                                      max_depth=50, 
                                      min_samples_leaf=3, 
                                      verbose=0, 
                                      random_state = 1)
    rf_model_50.fit(X_train_50,y_train)
    
    y_pred_proba_50 = rf_model_50.predict_proba(X_test_50)[:,1]
    
    # generate roc auc score and append to the list 
    roc_auc_50 = roc_auc_score(y_test, y_pred_proba_50)
    roc_auc_list_50.append(roc_auc_50)

In [33]:
roc_auc_list_50

[0.7937500000000001,
 0.7995326528102324,
 0.7763685700802265,
 0.7519466073414905,
 0.798076923076923,
 0.7977950310559008,
 0.7865900383141762,
 0.7864129786472623,
 0.7885695187165775,
 0.7847777222777222]

### report auc stats for 50 feature model

In [34]:
print('mean auc for 10 folds (50 features) is {:.3f}'.format(np.mean(roc_auc_list_50)))
print('std auc for 10 folds (50 features) is {:.3f}'.format(np.std(roc_auc_list_50)))
print('max auc for 10 folds (50 features) is {:.3f}'.format(np.max(roc_auc_list_50)))
print('min auc for 10 folds (50 features) is {:.3f}'.format(np.min(roc_auc_list_50)))

mean auc for 10 folds (50 features) is 0.786
std auc for 10 folds (50 features) is 0.013
max auc for 10 folds (50 features) is 0.800
min auc for 10 folds (50 features) is 0.752
